In [64]:
import pickle
with open('/root/nian/mollm_results/gcu/zgca,gpt-5-minimal/mols/gcu_mlp_1010_42.pkl','rb') as f:
    a = pickle.load(f)
all_mols = a['all_mols']


In [66]:
len(all_mols)

45

In [67]:
all_mols[10][0].constraints


{'status': 'failed',
 'debug_log': 'testcase0 -> Passed\ntestcase1 -> Passed\ntestcase2 -> Passed\ntestcase3 -> Passed\ntestcase4 -> Passed\ntestcase5 -> Passed\ntestcase6 -> Passed\ntestcase7 -> MLP Test Failed! MisMatch at Index : [0, 596]. CPU vs GCU : 2.42433 vs 2.42558\ntestcase8 -> Passed\ntestcase9 -> Passed\n',
 'time_comparison': [{'name': 'GCU_MLP0',
   'avg': '113.8 us',
   'best': '23.52 us'},
  {'name': 'GCU_MLP1', 'avg': '430.43 us', 'best': '149.23 us'},
  {'name': 'GCU_MLP2', 'avg': '74540.0 us', 'best': '1790.0 us'},
  {'name': 'GCU_MLP3', 'avg': '101780.0 us', 'best': '7600.0 us'},
  {'name': 'GCU_MLP4', 'avg': '28910.0 us', 'best': '3830.0 us'},
  {'name': 'GCU_MLP5', 'avg': '1580000.0 us', 'best': '40460.0 us'},
  {'name': 'GCU_MLP6', 'avg': '206890.0 us', 'best': '16350.0 us'},
  {'name': 'GCU_MLP8', 'avg': '77440.0 us', 'best': '6810.0 us'},
  {'name': 'GCU_MLP9', 'avg': '879060.0 us', 'best': '43590.0 us'}]}

In [23]:
a = sorted(a,key=lambda x:x.total,reverse=True)
a

In [30]:
a[2].total

90.66

In [27]:
print(a[1].value)


#include <tops/tops_runtime.h>
#include <tops.h>
#include <stdint.h>

using namespace tops;

static __host__ __forceinline__ int div_up_host(int a, int b) { return (a + b - 1) / b; }

// 改进点基于 parent#0：
// - 引入行间双缓冲+预取：对相邻输出行的输入3行集合做滚动复用（仅替换退出的那一行），减少DMA次数。
// - 对 dilation==1 的常见路径，构建“3行滑窗寄存器缓存”，在水平移动时只搬一组新列（以标量快速路径实现），减少随机访存判断。
// - 输出行仍整行写回；每线程专注一个通道集合。
// - 权重向量广播，常量化。

__global__ void dwconv3x3_row_rolling_prefetch_kernel(
    int C, int IH, int IW, int stride, int padding, int dilation,
    float* __restrict inp_g,
    float* __restrict weight_g,
    float* __restrict out_g)
{
    const int KH = 3, KW = 3;
    const int OH = (IH + 2 * padding - dilation * (KH - 1) - 1) / stride + 1;
    const int OW = (IW + 2 * padding - dilation * (KW - 1) - 1) / stride + 1;

    const int tid = threadIdx.x;
    const int tnum = blockDim.x;
    const int grid_threads = gridDim.x * tnum;

    const int V = tops::vlength<vfloat>(); // 32
    for (int c = blockIdx.x * tnum + tid; c < C; c += grid_t

In [37]:
a[2].constraints

{'status': 'success',
 'debug_log': 'testcase0 -> Passed\ntestcase1 -> Passed\ntestcase2 -> Passed\ntestcase3 -> Passed\ntestcase4 -> Passed\ntestcase5 -> Passed\ntestcase6 -> Passed\ntestcase7 -> Passed\ntestcase8 -> Passed\ntestcase9 -> Passed\n',
 'time_comparison': [{'name': 'GCU_CONV2D0',
   'avg': '24.85 us',
   'best': '20.06 us'},
  {'name': 'GCU_CONV2D1', 'avg': '86.22 us', 'best': '82.15 us'},
  {'name': 'GCU_CONV2D2', 'avg': '234.44 us', 'best': '195.95 us'},
  {'name': 'GCU_CONV2D3', 'avg': '2960.0 us', 'best': '2960.0 us'},
  {'name': 'GCU_CONV2D4', 'avg': '5390.0 us', 'best': '5390.0 us'},
  {'name': 'GCU_CONV2D5', 'avg': '315.81 us', 'best': '262.91 us'},
  {'name': 'GCU_CONV2D6', 'avg': '296.6 us', 'best': '227.96 us'},
  {'name': 'GCU_CONV2D7', 'avg': '2140.0 us', 'best': '1860.0 us'},
  {'name': 'GCU_CONV2D8', 'avg': '1050.0 us', 'best': '1050.0 us'},
  {'name': 'GCU_CONV2D9', 'avg': '43990.0 us', 'best': '43990.0 us'}]}

In [38]:
import re
scores = []
for item in a[2].constraints['time_comparison']:
    # 用正则提取数字部分
    avg = float(re.findall(r"[\d.]+", item['avg'])[0])
    best = float(re.findall(r"[\d.]+", item['best'])[0])
    score = best / avg
    scores.append(score)
    print(f"{item['name']}: best/avg = {score:.4f}")

mean_score = sum(scores) / len(scores)
print(f"\nOverall average score = {mean_score:.4f}")

GCU_CONV2D0: best/avg = 0.8072
GCU_CONV2D1: best/avg = 0.9528
GCU_CONV2D2: best/avg = 0.8358
GCU_CONV2D3: best/avg = 1.0000
GCU_CONV2D4: best/avg = 1.0000
GCU_CONV2D5: best/avg = 0.8325
GCU_CONV2D6: best/avg = 0.7686
GCU_CONV2D7: best/avg = 0.8692
GCU_CONV2D8: best/avg = 1.0000
GCU_CONV2D9: best/avg = 1.0000

Overall average score = 0.9066


In [39]:
a[2].total

90.66

In [31]:
a[2].constraints

{'status': 'success',
 'debug_log': 'testcase0 -> Passed\ntestcase1 -> Passed\ntestcase2 -> Passed\ntestcase3 -> Passed\ntestcase4 -> Passed\ntestcase5 -> Passed\ntestcase6 -> Passed\ntestcase7 -> Passed\ntestcase8 -> Passed\ntestcase9 -> Passed\n',
 'time_comparison': [{'name': 'GCU_CONV2D0',
   'avg': '24.85 us',
   'best': '20.06 us'},
  {'name': 'GCU_CONV2D1', 'avg': '86.22 us', 'best': '82.15 us'},
  {'name': 'GCU_CONV2D2', 'avg': '234.44 us', 'best': '195.95 us'},
  {'name': 'GCU_CONV2D3', 'avg': '2960.0 us', 'best': '2960.0 us'},
  {'name': 'GCU_CONV2D4', 'avg': '5390.0 us', 'best': '5390.0 us'},
  {'name': 'GCU_CONV2D5', 'avg': '315.81 us', 'best': '262.91 us'},
  {'name': 'GCU_CONV2D6', 'avg': '296.6 us', 'best': '227.96 us'},
  {'name': 'GCU_CONV2D7', 'avg': '2140.0 us', 'best': '1860.0 us'},
  {'name': 'GCU_CONV2D8', 'avg': '1050.0 us', 'best': '1050.0 us'},
  {'name': 'GCU_CONV2D9', 'avg': '43990.0 us', 'best': '43990.0 us'}]}

In [60]:
a[10].constraints

KeyError: 10

In [166]:
import re
import pickle
root = '/root/nian/mollm_results/gcu/zgca,gpt-5-minimal/mols/gcu_mlp_1010_42.pkl'
with open(root,'rb') as f:
    a = pickle.load(f)

all_mols = a['all_mols']

for mol, _ in reversed(all_mols):
    if mol.constraints['status'] == "success":
        # Step 2: 建立基准 best_time 字典
        print(mol.constraints)
        best_times = {}
        for d in mol.constraints["time_comparison"]:
            name = d["name"]
            best_us = float(re.findall(r"[\d.]+", d["best"])[0])
            best_times[name] = best_us
        break
print('best times:',best_times)

goal = all_mols[0][0].property_list[0]
print(goal)
for mol, _ in all_mols:
    con = mol.constraints

    # 没有 time_comparison → 0分
    if not con or "time_comparison" not in con:
        mol.total = 0.0
        mol.property[goal] = 100
        mol.scores = [1.0]
        continue

    # 把当前 item 的 time_comparison 转成字典形式方便查找
    item_times = {}
    for d in con["time_comparison"]:
        name = d.get("name")
        avg_str = d.get("avg")
        if name and avg_str:
            avg_us = float(re.findall(r"[\d.]+", avg_str)[0])
            item_times[name] = avg_us

    # 按 best_times 为主来循环
    scores = []
    for name, ref_best in best_times.items():
        if name not in item_times:
            # 这个 name 测试缺失（例如 MLP7）→ 记 0 分
            scores.append(0.0)
            continue
        
        avg_us = item_times[name]
        scores.append(ref_best / avg_us*100)
    mol.total = sum(scores) / len(scores) 
    mol.property[goal] = mol.total
    mol.scores = [1 - mol.total/100]

print("✅ 已根据最新基准重算完成。示例：")
kkk = sorted(all_mols,key=lambda x:x[0].total,reverse=True)
for i, (mol, _) in enumerate(kkk[:5]):  # 打印后5个看看
    print(f"{i}: total = {mol.total:.4f}")

{'status': 'success', 'debug_log': 'testcase0 -> Passed\ntestcase1 -> Passed\ntestcase2 -> Passed\ntestcase3 -> Passed\ntestcase4 -> Passed\ntestcase5 -> Passed\ntestcase6 -> Passed\ntestcase7 -> Passed\ntestcase8 -> Passed\ntestcase9 -> Passed\n', 'time_comparison': [{'name': 'GCU_MLP0', 'avg': '109.42 us', 'best': '23.52 us'}, {'name': 'GCU_MLP1', 'avg': '344.28 us', 'best': '149.23 us'}, {'name': 'GCU_MLP2', 'avg': '43190.0 us', 'best': '1790.0 us'}, {'name': 'GCU_MLP3', 'avg': '58150.0 us', 'best': '7600.0 us'}, {'name': 'GCU_MLP4', 'avg': '27120.0 us', 'best': '3830.0 us'}, {'name': 'GCU_MLP5', 'avg': '753020.0 us', 'best': '40460.0 us'}, {'name': 'GCU_MLP6', 'avg': '115730.0 us', 'best': '16350.0 us'}, {'name': 'GCU_MLP7', 'avg': '5370000.0 us', 'best': '106370.0 us'}, {'name': 'GCU_MLP8', 'avg': '61540.0 us', 'best': '6810.0 us'}, {'name': 'GCU_MLP9', 'avg': '454180.0 us', 'best': '43590.0 us'}]}
best times: {'GCU_MLP0': 23.52, 'GCU_MLP1': 149.23, 'GCU_MLP2': 1790.0, 'GCU_MLP3':

In [167]:
a['all_mols'] = all_mols
with open(root,'wb') as f:
    pickle.dump(a, f)

In [165]:
all_mols[kkk[0][1]-1][0].total

15.727266198750169

In [ ]:
def fix_results(item):
    goal = item.property_list[0]
    print(goal)
    mol = item
    con = mol.constraints

    # 没有 time_comparison → 0分
    if not con or "time_comparison" not in con:
        mol.total = 0.0
        mol.property[goal] = 100
        mol.scores = [1.0]
        return mol

    # 把当前 item 的 time_comparison 转成字典形式方便查找
    item_times = {}
    for d in con["time_comparison"]:
        name = d.get("name")
        avg_str = d.get("avg")
        if name and avg_str:
            avg_us = float(re.findall(r"[\d.]+", avg_str)[0])
            item_times[name] = avg_us

    # 按 best_times 为主来循环
    scores = []
    for name, ref_best in best_times.items():
        if name not in item_times:
            # 这个 name 测试缺失（例如 MLP7）→ 记 0 分
            scores.append(0.0)
            continue
        
        avg_us = item_times[name]
        scores.append(ref_best / avg_us*100)
    mol.total = sum(scores) / len(scores) 
    mol.property[goal] = mol.total
    mol.scores = [1 - mol.total/100]
    return mol

In [171]:
a.keys()

dict_keys(['history', 'init_pops', 'final_pops', 'all_mols', 'properties', 'evaluation', 'running_time'])

In [ ]:
import json

data = {
    "title": "可落地的高性能经验",
    "author": "姜诗蕴",
    "content": "这是一段测试文字。"
}

# 正确写法：ensure_ascii=False
with open("example.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
